In [6]:
import math
import numpy as np
from cmaes import CMA

## Objective Function

In [30]:
def ackley(x1, x2):
    # https://www.sfu.ca/~ssurjano/ackley.html
#     return (
#         -20 * math.exp(-0.2 * math.sqrt(0.5 * (x1 ** 2 + x2 ** 2)))
#         - math.exp(0.5 * (math.cos(2 * math.pi * x1) + math.cos(2 * math.pi * x2)))
#         + math.e + 20
#     )
    return x1 - x2

## IPOP-CMA-ES

In [31]:
bounds = np.array([[-32.768, 32.768], [-32.768, 32.768]])
lower_bounds, upper_bounds = bounds[:, 0], bounds[:, 1]

mean = lower_bounds + (np.random.rand(2) * (upper_bounds - lower_bounds))
sigma = 32.768 * 2 / 5  # 1/5 of the domain width
optimizer = CMA(mean=mean, sigma=sigma, bounds=bounds, seed=0)

for generation in range(5):
    solutions = []
    for _ in range(optimizer.population_size):
        x = optimizer.ask()
        value = ackley(x[0], x[1])
        solutions.append((x, value))
        print(f"#{generation} {value} (x1={x[0]}, x2 = {x[1]})")
    optimizer.tell(solutions)

    if optimizer.should_stop():
        # popsize multiplied by 2 (or 3) before each restart.
        popsize = optimizer.population_size * 2
        mean = lower_bounds + (np.random.rand(2) * (upper_bounds - lower_bounds))
        optimizer = CMA(mean=mean, sigma=sigma, population_size=popsize)
        print(f"Restart CMA-ES with popsize={popsize}")

#0 -9.506258956337032 (x1=12.019199962146146, x2 = 21.525458918483178)
#0 9.90472741171622 (x1=13.375869141572434, x2 = 3.471141729856214)
#0 -12.946237197102539 (x1=1.3504119592701844, x2 = 14.296649156372723)
#0 -34.11781212272869 (x1=-12.455497081149742, x2 = 21.662315041578942)
#0 -19.002850409021658 (x1=-1.1275132758745485, x2 = 17.87533713314711)
#0 -25.938837283412678 (x1=-5.284782782680377, x2 = 20.6540545007323)
#1 -45.05433827104556 (x1=-23.046209622643474, x2 = 22.008128648402085)
#1 -27.2950468968164 (x1=-14.266483594029971, x2 = 13.02856330278643)
#1 -8.284327130032754 (x1=14.2863200346744, x2 = 22.570647164707154)
#1 -34.093427022578 (x1=-18.814470615065964, x2 = 15.278956407512034)
#1 -43.62658186655756 (x1=-32.75115427239435, x2 = 10.875427594163208)
#1 -29.63444895163712 (x1=-10.366258476065335, x2 = 19.26819047557178)
#2 -46.22950266591718 (x1=-30.548724887116784, x2 = 15.680777778800401)
#2 -50.188170293501756 (x1=-28.343885846202927, x2 = 21.844284447298833)
#2 -45.

## BIPOP-CMA-ES

In [32]:
bounds = np.array([[-10, 10], [-10, 10]])
lower_bounds, upper_bounds = bounds[:, 0], bounds[:, 1]

mean = lower_bounds + (np.random.rand(2) * (upper_bounds - lower_bounds))
sigma = 10 * 2 / 5  # 1/5 of the domain width
optimizer = CMA(mean=mean, sigma=sigma, bounds=bounds, seed=0)

n_restarts = 0  # A small restart doesn't count in the n_restarts
small_n_eval, large_n_eval = 0, 0
popsize0 = optimizer.population_size
inc_popsize = 2

# Initial run is with "normal" population size; it is
# the large population before first doubling, but its
# budget accounting is the same as in case of small
# population.
poptype = "small"

for generation in range(10):
    solutions = []
    for _ in range(optimizer.population_size):
        x = optimizer.ask()
        value = ackley(x[0], x[1])
        solutions.append((x, value))
        print(f"#{generation} {value} (x1={x[0]}, x2 = {x[1]})")
    optimizer.tell(solutions)

    if optimizer.should_stop():
        n_eval = optimizer.population_size * optimizer.generation
        if poptype == "small":
            small_n_eval += n_eval
        else:  # poptype == "large"
            large_n_eval += n_eval

        if small_n_eval < large_n_eval:
            poptype = "small"
            popsize_multiplier = inc_popsize ** n_restarts
            popsize = math.floor(
                popsize0 * popsize_multiplier ** (np.random.uniform() ** 2)
            )
        else:
            poptype = "large"
            n_restarts += 1
            popsize = popsize0 * (inc_popsize ** n_restarts)

        mean = lower_bounds + (np.random.rand(2) * (upper_bounds - lower_bounds))
        optimizer = CMA(
            mean=mean,
            sigma=sigma,
            bounds=bounds,
            population_size=popsize,
        )
        print("Restart CMA-ES with popsize={} ({})".format(popsize, poptype))

#0 0.528235329174902 (x1=6.139172164391592, x2 = 5.61093683521669)
#0 -6.445325647637464 (x1=2.4692938738940797, x2 = 8.914619521531543)
#0 -4.054158803236985 (x1=4.645256496422147, x2 = 8.699415299659131)
#0 -6.182454387120107 (x1=-0.07878898960467229, x2 = 6.103665397515435)
#0 -4.424703273866822 (x1=2.126684999229745, x2 = 6.551388273096567)
#0 4.045373043414878 (x1=7.747958699742153, x2 = 3.702585656327275)
#1 -6.400856242843519 (x1=1.1385767723684803, x2 = 7.5394330152119995)
#1 -7.087231787295645 (x1=2.2297537165380965, x2 = 9.316985503833742)
#1 -1.241512249257283 (x1=-0.7644744447685636, x2 = 0.47703780448871935)
#1 -4.481722219554313 (x1=3.022958160353115, x2 = 7.504680379907428)
#1 -4.174580515222187 (x1=2.0071754058262923, x2 = 6.18175592104848)
#1 -0.5558489938637972 (x1=1.032263421209032, x2 = 1.5881124150728292)
#2 -6.460073379344831 (x1=3.383394334923237, x2 = 9.843467714268067)
#2 -4.083503053574978 (x1=5.586024069317931, x2 = 9.669527122892909)
#2 -8.125121255404373 (x

In [33]:
solutions

[(array([-8.48988342,  9.66471482]), -18.154598237888205),
 (array([-9.38391212,  7.50638801]), -16.89030012962114),
 (array([-6.78880738,  9.19174201]), -15.980549385937206),
 (array([-6.56286166,  9.24397185]), -15.806833515286858),
 (array([-8.80715699,  6.91442432]), -15.721581305921411),
 (array([-6.21155628,  3.78696202]), -9.998518303423223)]